In [1]:
if 0 :
    %matplotlib inline
else :
    %matplotlib notebook

#  Import  libraries

In [2]:
import sys
import os
module_path = os.path.abspath('.') +"\\_scripts"
print(module_path)

if module_path not in sys.path:
    sys.path.append(module_path)
from _00_Import_packages_git3 import *

C:\Users\ng947ac\Documents\Daniel\jupyter\SoSTrades\jupyter_doc\_scripts
C:\Users\ng947ac\Documents\Daniel\git\workspace
C:\Users\ng947ac\Documents\Daniel\git\workspace\sosgemseo\src\
C:\Users\ng947ac\Documents\Daniel\git\workspace\sostrades-core


In [3]:
from platform import python_version

print(python_version())

3.9.7


In [4]:
import numpy
print(numpy.__version__)

1.20.3


In [5]:
import scipy
print(scipy.__version__)

1.7.1


In [6]:
import pandas
print(pandas.__version__)

1.3.0


In [7]:
from numpy import array
import pandas as pd
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine
from numpy.testing import assert_array_equal, assert_array_almost_equal  # @UnresolvedImport
import os
from gemseo.core.mdo_scenario import MDOScenario

# 1) Set up

In [8]:
#from sos_trades_core.sos_wrapping.test_discs.sellar import Sellar1
from sos_trades_core.execution_engine.execution_engine import ExecutionEngine

In [9]:
study_name = 'optim'
ns = f'{study_name}'
sc_name = "SobOptimScenario"

In [10]:
repo = 'sos_trades_core.sos_processes.test'
proc_name = 'test_sobieski_opt'

# 2) Sobieski optim builder

In [11]:
exec_eng = ExecutionEngine(study_name)
factory = exec_eng.factory
builder = factory.get_builder_from_process(repo=repo,mod_id=proc_name)
exec_eng.factory.set_builders_to_coupling_builder(builder)

In [12]:
exec_eng.configure() 

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling optim is set to MDAJacobi


In [13]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview optim
|_ optim
	|_ SobOptimScenario
		|_ struct
		|_ aero
		|_ prop
		|_ mission


'Nodes representation for Treeview optim\n|_ optim\n\t|_ SobOptimScenario\n\t\t|_ struct\n\t\t|_ aero\n\t\t|_ prop\n\t\t|_ mission'

In [14]:
exec_eng.display_treeview_nodes(True)

INFO:SoS.EE:Nodes representation for Treeview optim
|_ optim
	-> acceleration
	-> authorize_self_coupled_disciplines
	-> cache_file_path
	-> cache_type
	-> chain_linearize
	-> debug_mode
	-> epsilon0
	-> group_mda_disciplines
	-> linear_solver_MDA
	-> linear_solver_MDA_options
	-> linear_solver_MDA_preconditioner
	-> linear_solver_MDO
	-> linear_solver_MDO_options
	-> linear_solver_MDO_preconditioner
	-> linearization_mode
	-> max_mda_iter
	-> n_processes
	-> n_subcouplings_parallel
	-> relax_factor
	-> sub_mda_class
	-> tolerance
	-> tolerance_gs
	-> use_lu_fact
	-> warm_start
	-> warm_start_threshold
	<- residuals_history
	|_ SobOptimScenario
		-> algo
		-> algo_options
		-> cache_file_path
		-> cache_type
		-> debug_mode
		-> design_space
		-> differentiation_method
		-> eq_constraints
		-> eval_jac
		-> eval_mode
		-> execute_at_xopt
		-> formulation
		-> ineq_constraints
		-> linearization_mode
		-> max_iter
		-> maximize_objective
		-> objective_name
		-> parallel_options
		-> x_

'Nodes representation for Treeview optim\n|_ optim\n\t-> acceleration\n\t-> authorize_self_coupled_disciplines\n\t-> cache_file_path\n\t-> cache_type\n\t-> chain_linearize\n\t-> debug_mode\n\t-> epsilon0\n\t-> group_mda_disciplines\n\t-> linear_solver_MDA\n\t-> linear_solver_MDA_options\n\t-> linear_solver_MDA_preconditioner\n\t-> linear_solver_MDO\n\t-> linear_solver_MDO_options\n\t-> linear_solver_MDO_preconditioner\n\t-> linearization_mode\n\t-> max_mda_iter\n\t-> n_processes\n\t-> n_subcouplings_parallel\n\t-> relax_factor\n\t-> sub_mda_class\n\t-> tolerance\n\t-> tolerance_gs\n\t-> use_lu_fact\n\t-> warm_start\n\t-> warm_start_threshold\n\t<- residuals_history\n\t|_ SobOptimScenario\n\t\t-> algo\n\t\t-> algo_options\n\t\t-> cache_file_path\n\t\t-> cache_type\n\t\t-> debug_mode\n\t\t-> design_space\n\t\t-> differentiation_method\n\t\t-> eq_constraints\n\t\t-> eval_jac\n\t\t-> eval_mode\n\t\t-> execute_at_xopt\n\t\t-> formulation\n\t\t-> ineq_constraints\n\t\t-> linearization_mode\n

# 3) Design Space

In [15]:
dspace_dict = {'variable': ['z', 'x_1', 'x_2', 'x_3'],
               'value': [
                        [0.05, 45000.0,1.6,5.5,55.0,1000.0], 
                        [0.25, 1.0], 
                        [1.0],
                        [0.5]],
               'lower_bnd': [
                        [0.01,30000.0,1.4,2.5,40.0,500.0], 
                        [0.1,0.75], 
                        [0.75],
                        [0.1]],
               'upper_bnd': [
                        [0.09,60000.0,1.8,8.5,70.0,1500.0], 
                        [0.4,1.25], 
                        [1.25],
                        [1.0]],
               'enable_variable': [True, True, True, True],
               'activated_elem': [[True, True, True, True, True, True], [True, True], [True], [True]]}

In [16]:
dspace = pd.DataFrame(dspace_dict)
dspace

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,z,"[0.05, 45000.0, 1.6, 5.5, 55.0, 1000.0]","[0.01, 30000.0, 1.4, 2.5, 40.0, 500.0]","[0.09, 60000.0, 1.8, 8.5, 70.0, 1500.0]",True,"[True, True, True, True, True, True]"
1,x_1,"[0.25, 1.0]","[0.1, 0.75]","[0.4, 1.25]",True,"[True, True]"
2,x_2,[1.0],[0.75],[1.25],True,[True]
3,x_3,[0.5],[0.1],[1.0],True,[True]


In [17]:
#dspace.to_dict('list')

In [18]:
dspace_coupl_dict = {'variable': ['y_14','y_32','y_31','y_24','y_34','y_23','y_21','y_12'],
               'value': [
                        [50606.9741711,7306.20262124], 
                        [0.50279625], 
                        [6354.32430691],
                        [4.15006276],
                        [1.10754577],
                        [12194.2671934],
                        [50606.9741711],
                        [50606.9742,0.95]
                        ],
               'lower_bnd':  [
                        [24850.0,-7700.0], 
                        [0.235], 
                        [2960.0],
                        [0.44],
                        [0.44],
                        [3365.0],
                        [24850.0],
                        [24850.0,0.45]
                        ],
               'upper_bnd':   [
                        [77100.0,45000.0], 
                        [0.795], 
                        [10185.0],
                        [11.13],
                        [1.98],
                        [26400.0],
                        [77250.0],
                        [77250.0,1.5]
                        ],
               'enable_variable': [True, True, True, True, True, True, True, True],
               'activated_elem': [[True, True], [True], [True], [True], [True], [True], [True],[True, True]]}

In [19]:
dspace_coupl = pd.DataFrame(dspace_coupl_dict)
dspace_coupl

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,y_14,"[50606.9741711, 7306.20262124]","[24850.0, -7700.0]","[77100.0, 45000.0]",True,"[True, True]"
1,y_32,[0.50279625],[0.235],[0.795],True,[True]
2,y_31,[6354.32430691],[2960.0],[10185.0],True,[True]
3,y_24,[4.15006276],[0.44],[11.13],True,[True]
4,y_34,[1.10754577],[0.44],[1.98],True,[True]
5,y_23,[12194.2671934],[3365.0],[26400.0],True,[True]
6,y_21,[50606.9741711],[24850.0],[77250.0],True,[True]
7,y_12,"[50606.9742, 0.95]","[24850.0, 0.45]","[77250.0, 1.5]",True,"[True, True]"


In [20]:
# merge dataframe (add lines)
frames = [dspace,dspace_coupl]
dspace_all = pd.concat(frames)
dspace_all

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,z,"[0.05, 45000.0, 1.6, 5.5, 55.0, 1000.0]","[0.01, 30000.0, 1.4, 2.5, 40.0, 500.0]","[0.09, 60000.0, 1.8, 8.5, 70.0, 1500.0]",True,"[True, True, True, True, True, True]"
1,x_1,"[0.25, 1.0]","[0.1, 0.75]","[0.4, 1.25]",True,"[True, True]"
2,x_2,[1.0],[0.75],[1.25],True,[True]
3,x_3,[0.5],[0.1],[1.0],True,[True]
0,y_14,"[50606.9741711, 7306.20262124]","[24850.0, -7700.0]","[77100.0, 45000.0]",True,"[True, True]"
1,y_32,[0.50279625],[0.235],[0.795],True,[True]
2,y_31,[6354.32430691],[2960.0],[10185.0],True,[True]
3,y_24,[4.15006276],[0.44],[11.13],True,[True]
4,y_34,[1.10754577],[0.44],[1.98],True,[True]
5,y_23,[12194.2671934],[3365.0],[26400.0],True,[True]


In [21]:
# convert dataframe to dict
dspace_all_dict = dspace_all.to_dict('list')

# 4) Optim and Sobieski inputs

In [22]:
#-- set up disciplines in Scenario
disc_dict = {}
# Optim inputs
disc_dict[f'{ns}.{sc_name}.max_iter'] = 10
# SLSQP, NLOPT_SLSQP
disc_dict[f'{ns}.{sc_name}.algo'] = "SLSQP"
disc_dict[f'{ns}.{sc_name}.design_space'] = dspace_all
#disc_dict[f'{ns}.{sc_name}.design_space'] = dspace # does not work!
disc_dict[f'{ns}.{sc_name}.formulation'] = 'MDF'
disc_dict[f'{ns}.{sc_name}.objective_name'] = 'y_4'
disc_dict[f'{ns}.{sc_name}.maximize_objective'] = True
disc_dict[f'{ns}.{sc_name}.ineq_constraints'] = [f'g_1', f'g_2', f'g_3']
disc_dict[f'{ns}.{sc_name}.algo_options'] = {"ftol_rel": 1e-10,
                                                            "ineq_tolerance": 2e-3,
                                                            "normalize_design_space": True}
disc_dict[f'{ns}.tolerance'] = 1e-10
disc_dict[f'{ns}.max_mda_iter'] = 50
disc_dict[f'{ns}.warm_start'] = True
disc_dict[f'{ns}.use_lu_fact'] = True
disc_dict

{'optim.SobOptimScenario.max_iter': 10,
 'optim.SobOptimScenario.algo': 'SLSQP',
 'optim.SobOptimScenario.design_space':   variable                                    value  \
 0        z  [0.05, 45000.0, 1.6, 5.5, 55.0, 1000.0]   
 1      x_1                              [0.25, 1.0]   
 2      x_2                                    [1.0]   
 3      x_3                                    [0.5]   
 0     y_14           [50606.9741711, 7306.20262124]   
 1     y_32                             [0.50279625]   
 2     y_31                          [6354.32430691]   
 3     y_24                             [4.15006276]   
 4     y_34                             [1.10754577]   
 5     y_23                          [12194.2671934]   
 6     y_21                          [50606.9741711]   
 7     y_12                       [50606.9742, 0.95]   
 
                                 lower_bnd  \
 0  [0.01, 30000.0, 1.4, 2.5, 40.0, 500.0]   
 1                             [0.1, 0.75]   
 2          

In [23]:
#exec_eng.dm.set_values_from_dict(disc_dict)
exec_eng.load_study_from_input_dict(disc_dict)

INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling optim is set to MDAJacobi


In [24]:
# discipline inputs

In [25]:
values_dict = {
                  f'{ns}.{sc_name}.z': [0.05,45000,1.6,5.5,55.,1000],
                  f'{ns}.{sc_name}.y_14': [50606.9,7306.20],
                  f'{ns}.{sc_name}.y_24':  [4.15], 
                  f'{ns}.{sc_name}.y_34': [1.10], 
                  f'{ns}.{sc_name}.x_1': [0.25,1.0],
                  f'{ns}.{sc_name}.y_21':  [50606.9], 
                  f'{ns}.{sc_name}.y_31': [6354.32], 
                  f'{ns}.{sc_name}.x_2': [1.0],
                  f'{ns}.{sc_name}.y_12':  [50606.9,0.95], 
                  f'{ns}.{sc_name}.y_32': [12194.2], 
                  f'{ns}.{sc_name}.x_3': [0.5],
                  f'{ns}.{sc_name}.y_23':  [12194.2], 
                  }
values_dict

{'optim.SobOptimScenario.z': [0.05, 45000, 1.6, 5.5, 55.0, 1000],
 'optim.SobOptimScenario.y_14': [50606.9, 7306.2],
 'optim.SobOptimScenario.y_24': [4.15],
 'optim.SobOptimScenario.y_34': [1.1],
 'optim.SobOptimScenario.x_1': [0.25, 1.0],
 'optim.SobOptimScenario.y_21': [50606.9],
 'optim.SobOptimScenario.y_31': [6354.32],
 'optim.SobOptimScenario.x_2': [1.0],
 'optim.SobOptimScenario.y_12': [50606.9, 0.95],
 'optim.SobOptimScenario.y_32': [12194.2],
 'optim.SobOptimScenario.x_3': [0.5],
 'optim.SobOptimScenario.y_23': [12194.2]}

In [26]:
#exec_eng.dm.set_values_from_dict(values_dict)
exec_eng.load_study_from_input_dict(values_dict)

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling optim is set to MDAJacobi


In [27]:
exec_eng.configure()

INFO:SoS.EE:configuring ...
INFO:SoS.EE.Coupling:The MDA solver of the Coupling optim is set to MDAJacobi


In [28]:
exec_eng.dm.get_data_dict_values()

{'optim.sub_mda_class': 'MDAJacobi',
 'optim.max_mda_iter': 50,
 'optim.n_processes': 1,
 'optim.chain_linearize': False,
 'optim.tolerance': 1e-10,
 'optim.use_lu_fact': True,
 'optim.warm_start': True,
 'optim.acceleration': 'm2d',
 'optim.warm_start_threshold': -1,
 'optim.n_subcouplings_parallel': 1,
 'optim.tolerance_gs': 10.0,
 'optim.relax_factor': 0.99,
 'optim.epsilon0': 1e-06,
 'optim.linear_solver_MDO': 'GMRES',
 'optim.linear_solver_MDO_preconditioner': 'None',
 'optim.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'optim.linear_solver_MDA': 'GMRES',
 'optim.linear_solver_MDA_preconditioner': 'None',
 'optim.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'optim.group_mda_disciplines': False,
 'optim.authorize_self_coupled_disciplines': False,
 'optim.linearization_mode': 'auto',
 'optim.cache_type': 'SimpleCache',
 'optim.cache_file_path': None,
 'optim.debug_mode': '',
 'optim.SobOptimScenario.max_iter': 10,
 'optim.SobOptimScenario.ineq_con

In [29]:
#opt_disc.get_sos_disciplines()

In [30]:
#help(opt_disc)

In [31]:
#opt_disc.disciplines

In [32]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview optim
|_ optim
	|_ SobOptimScenario
		|_ struct
		|_ aero
		|_ prop
		|_ mission


'Nodes representation for Treeview optim\n|_ optim\n\t|_ SobOptimScenario\n\t\t|_ struct\n\t\t|_ aero\n\t\t|_ prop\n\t\t|_ mission'

In [33]:
exec_eng.display_treeview_nodes(True)

INFO:SoS.EE:Nodes representation for Treeview optim
|_ optim
	-> acceleration
	-> authorize_self_coupled_disciplines
	-> cache_file_path
	-> cache_type
	-> chain_linearize
	-> debug_mode
	-> epsilon0
	-> group_mda_disciplines
	-> linear_solver_MDA
	-> linear_solver_MDA_options
	-> linear_solver_MDA_preconditioner
	-> linear_solver_MDO
	-> linear_solver_MDO_options
	-> linear_solver_MDO_preconditioner
	-> linearization_mode
	-> max_mda_iter
	-> n_processes
	-> n_subcouplings_parallel
	-> relax_factor
	-> sub_mda_class
	-> tolerance
	-> tolerance_gs
	-> use_lu_fact
	-> warm_start
	-> warm_start_threshold
	<- residuals_history
	|_ SobOptimScenario
		-> algo
		-> algo_options
		-> cache_file_path
		-> cache_type
		-> debug_mode
		-> design_space
		-> differentiation_method
		-> eq_constraints
		-> eval_jac
		-> eval_mode
		-> execute_at_xopt
		-> formulation
		-> ineq_constraints
		-> linearization_mode
		-> max_iter
		-> maximize_objective
		-> objective_name
		-> parallel_options
		-> x_

'Nodes representation for Treeview optim\n|_ optim\n\t-> acceleration\n\t-> authorize_self_coupled_disciplines\n\t-> cache_file_path\n\t-> cache_type\n\t-> chain_linearize\n\t-> debug_mode\n\t-> epsilon0\n\t-> group_mda_disciplines\n\t-> linear_solver_MDA\n\t-> linear_solver_MDA_options\n\t-> linear_solver_MDA_preconditioner\n\t-> linear_solver_MDO\n\t-> linear_solver_MDO_options\n\t-> linear_solver_MDO_preconditioner\n\t-> linearization_mode\n\t-> max_mda_iter\n\t-> n_processes\n\t-> n_subcouplings_parallel\n\t-> relax_factor\n\t-> sub_mda_class\n\t-> tolerance\n\t-> tolerance_gs\n\t-> use_lu_fact\n\t-> warm_start\n\t-> warm_start_threshold\n\t<- residuals_history\n\t|_ SobOptimScenario\n\t\t-> algo\n\t\t-> algo_options\n\t\t-> cache_file_path\n\t\t-> cache_type\n\t\t-> debug_mode\n\t\t-> design_space\n\t\t-> differentiation_method\n\t\t-> eq_constraints\n\t\t-> eval_jac\n\t\t-> eval_mode\n\t\t-> execute_at_xopt\n\t\t-> formulation\n\t\t-> ineq_constraints\n\t\t-> linearization_mode\n

In [34]:
for key in exec_eng.dm.data_id_map:
    print("key", key)

key optim.sub_mda_class
key optim.max_mda_iter
key optim.n_processes
key optim.chain_linearize
key optim.tolerance
key optim.use_lu_fact
key optim.warm_start
key optim.acceleration
key optim.warm_start_threshold
key optim.n_subcouplings_parallel
key optim.tolerance_gs
key optim.relax_factor
key optim.epsilon0
key optim.linear_solver_MDO
key optim.linear_solver_MDO_preconditioner
key optim.linear_solver_MDO_options
key optim.linear_solver_MDA
key optim.linear_solver_MDA_preconditioner
key optim.linear_solver_MDA_options
key optim.group_mda_disciplines
key optim.authorize_self_coupled_disciplines
key optim.linearization_mode
key optim.cache_type
key optim.cache_file_path
key optim.debug_mode
key optim.SobOptimScenario.max_iter
key optim.SobOptimScenario.ineq_constraints
key optim.SobOptimScenario.eq_constraints
key optim.SobOptimScenario.algo
key optim.SobOptimScenario.design_space
key optim.SobOptimScenario.formulation
key optim.SobOptimScenario.maximize_objective
key optim.SobOptimScen

In [35]:
exec_eng.dm.get_data_dict_values()

{'optim.sub_mda_class': 'MDAJacobi',
 'optim.max_mda_iter': 50,
 'optim.n_processes': 1,
 'optim.chain_linearize': False,
 'optim.tolerance': 1e-10,
 'optim.use_lu_fact': True,
 'optim.warm_start': True,
 'optim.acceleration': 'm2d',
 'optim.warm_start_threshold': -1,
 'optim.n_subcouplings_parallel': 1,
 'optim.tolerance_gs': 10.0,
 'optim.relax_factor': 0.99,
 'optim.epsilon0': 1e-06,
 'optim.linear_solver_MDO': 'GMRES',
 'optim.linear_solver_MDO_preconditioner': 'None',
 'optim.linear_solver_MDO_options': {'max_iter': 1000, 'tol': 1e-08},
 'optim.linear_solver_MDA': 'GMRES',
 'optim.linear_solver_MDA_preconditioner': 'None',
 'optim.linear_solver_MDA_options': {'max_iter': 1000, 'tol': 1e-08},
 'optim.group_mda_disciplines': False,
 'optim.authorize_self_coupled_disciplines': False,
 'optim.linearization_mode': 'auto',
 'optim.cache_type': 'SimpleCache',
 'optim.cache_file_path': None,
 'optim.debug_mode': '',
 'optim.SobOptimScenario.max_iter': 10,
 'optim.SobOptimScenario.ineq_con

In [36]:
exec_eng.dm.get_data('optim.SobOptimScenario.formulation')['value']

'MDF'

In [37]:
exec_eng.dm.get_data('optim.linear_solver_MDO')['value']

'GMRES'

In [38]:
exec_eng.dm.get_data('optim.sub_mda_class')['value']

'MDAJacobi'

In [39]:
exec_eng.dm.get_data('optim.tolerance')['value']

1e-10

In [40]:
exec_eng.dm.get_data('optim.max_mda_iter')['value']

50

In [41]:
exec_eng.dm.get_data('optim.warm_start')['value']

True

In [42]:
exec_eng.dm.get_data('optim.use_lu_fact')['value']

True

In [43]:
exec_eng.dm.get_data(f'{ns}.{sc_name}.algo_options')['value']

{'xtol_rel': 1e-09,
 'xtol_abs': 1e-09,
 'ftol_rel': 1e-10,
 'ftol_abs': 1e-09,
 'max_time': 0,
 'disp': 0,
 'eq_tolerance': 0.01,
 'ineq_tolerance': 0.002,
 'max_iter': 999,
 'normalize_design_space': True}

In [44]:
# retrieve discipline to check the result...
opt_disc = exec_eng.dm.get_disciplines_with_name("optim." + sc_name)[0]
opt_disc.get_disc_full_name()

'optim.SobOptimScenario'

In [45]:
# retrieve discipline to check the result...
#help(opt_disc)

In [46]:
#opt_disc.disciplines

In [47]:
exec_eng.display_treeview_nodes()

INFO:SoS.EE:Nodes representation for Treeview optim
|_ optim
	|_ SobOptimScenario
		|_ struct
		|_ aero
		|_ prop
		|_ mission


'Nodes representation for Treeview optim\n|_ optim\n\t|_ SobOptimScenario\n\t\t|_ struct\n\t\t|_ aero\n\t\t|_ prop\n\t\t|_ mission'

In [48]:
opt_disc.xdsmize(html_output=True, print_statuses=False,open_browser=False,outfilename='my_xdsm_mdf_sob.html')

INFO:gemseo.utils.xdsmizer:Generating HTML XDSM file in : my_xdsm_mdf_sob.html


# 5) Execute

In [49]:
print(exec_eng.execute())

INFO:SoS.EE:PROCESS EXECUTION optim STARTS...
INFO:gemseo.core.mdo_scenario: 
INFO:gemseo.core.mdo_scenario:*** Start MDO Scenario execution ***
INFO:gemseo.core.mdo_scenario:SobOptimScenario
   Disciplines: struct aero prop mission
   MDOFormulation: MDF
   Algorithm: None
INFO:SoS.EE.SoSOptimScenario:{'xtol_rel': 1e-09, 'xtol_abs': 1e-09, 'ftol_rel': 1e-10, 'ftol_abs': 1e-09, 'max_time': 0, 'disp': 0, 'eq_tolerance': 0.01, 'ineq_tolerance': 0.002, 'normalize_design_space': True}
INFO:gemseo.algos.opt.opt_lib:Optimization problem:
   Minimize: -optim.SobOptimScenario.y_4(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3)
   With respect to: optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3
   Subject to constraints:
      optim.SobOptimScenario.g_1(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3) <= 0.0
     

INFO:gemseo.mda.mda:	9	3.542673e-08
INFO:gemseo.mda.mda:	1	8.764956e-01
INFO:gemseo.mda.mda:	2	4.795709e-01
INFO:gemseo.mda.mda:	3	3.852500e-01
INFO:gemseo.mda.mda:	4	3.014612e-01
INFO:gemseo.mda.mda:	5	2.413705e-01
INFO:gemseo.mda.mda:	6	1.660039e-01
INFO:gemseo.mda.mda:	7	5.533888e-02
INFO:gemseo.mda.mda:	8	4.407176e-05
INFO:gemseo.mda.mda:	9	2.794148e-08
INFO:gemseo.mda.mda:	1	1.042939e+00
INFO:gemseo.mda.mda:	2	5.706396e-01
INFO:gemseo.mda.mda:	3	4.584102e-01
INFO:gemseo.mda.mda:	4	3.587137e-01
INFO:gemseo.mda.mda:	5	2.872101e-01
INFO:gemseo.mda.mda:	6	1.975232e-01
INFO:gemseo.mda.mda:	7	6.584310e-02
INFO:gemseo.mda.mda:	8	5.244451e-05
INFO:gemseo.mda.mda:	9	3.322216e-08
INFO:gemseo.mda.mda:	1	1.000320e+00
INFO:gemseo.mda.mda:	2	5.477313e-01
INFO:gemseo.mda.mda:	3	4.419671e-01
INFO:gemseo.mda.mda:	4	3.492353e-01
INFO:gemseo.mda.mda:	5	2.789424e-01
INFO:gemseo.mda.mda:	6	1.864248e-01
INFO:gemseo.mda.mda:	7	6.039596e-02
INFO:gemseo.mda.mda:	8	5.148384e-05
INFO:gemseo.mda.mda:	9	3.163

INFO:gemseo.mda.mda:	3	4.279479e-01
INFO:gemseo.mda.mda:	4	3.432176e-01
INFO:gemseo.mda.mda:	5	2.730887e-01
INFO:gemseo.mda.mda:	6	1.724346e-01
INFO:gemseo.mda.mda:	7	5.308673e-02
INFO:gemseo.mda.mda:	8	5.193331e-05
INFO:gemseo.mda.mda:	9	3.588817e-08
INFO:gemseo.mda.mda:	1	1.092152e+00
INFO:gemseo.mda.mda:	2	6.058017e-01
INFO:gemseo.mda.mda:	3	4.875552e-01
INFO:gemseo.mda.mda:	4	3.972789e-01
INFO:gemseo.mda.mda:	5	3.147072e-01
INFO:gemseo.mda.mda:	6	1.825075e-01
INFO:gemseo.mda.mda:	7	5.395015e-02
INFO:gemseo.mda.mda:	8	5.789310e-05
INFO:gemseo.mda.mda:	9	3.957953e-08
INFO:gemseo.mda.mda:	1	1.000002e+00
INFO:gemseo.mda.mda:	2	5.471488e-01
INFO:gemseo.mda.mda:	3	4.397350e-01
INFO:gemseo.mda.mda:	4	3.443680e-01
INFO:gemseo.mda.mda:	5	2.756710e-01
INFO:gemseo.mda.mda:	6	1.891944e-01
INFO:gemseo.mda.mda:	7	6.293454e-02
INFO:gemseo.mda.mda:	8	5.028313e-05
INFO:gemseo.mda.mda:	9	3.137649e-08
INFO:gemseo.mda.mda:	1	1.033969e+00
INFO:gemseo.mda.mda:	2	6.070100e-01
INFO:gemseo.mda.mda:	3	4.597

INFO:gemseo.mda.mda:	5	6.321406e+03
INFO:gemseo.mda.mda:	6	7.633531e+02
INFO:gemseo.mda.mda:	7	1.830024e+02
INFO:gemseo.mda.mda:	8	6.276206e+01
INFO:gemseo.mda.mda:	9	1.875483e+01
INFO:gemseo.mda.mda:	10	3.428548e-01
INFO:gemseo.mda.mda:	11	1.299375e-03
INFO:gemseo.mda.mda:	12	2.237411e-06
INFO:gemseo.mda.mda:	13	1.902079e-08
INFO:gemseo.mda.mda:	1	2.105029e+05
INFO:gemseo.mda.mda:	2	2.243831e+05
INFO:gemseo.mda.mda:	3	9.730586e+04
INFO:gemseo.mda.mda:	4	5.114868e+04
INFO:gemseo.mda.mda:	5	6.321422e+03
INFO:gemseo.mda.mda:	6	7.633584e+02
INFO:gemseo.mda.mda:	7	1.830029e+02
INFO:gemseo.mda.mda:	8	6.276229e+01
INFO:gemseo.mda.mda:	9	1.875491e+01
INFO:gemseo.mda.mda:	10	3.428531e-01
INFO:gemseo.mda.mda:	11	1.299366e-03
INFO:gemseo.mda.mda:	12	2.237411e-06
INFO:gemseo.mda.mda:	13	1.907226e-08
INFO:gemseo.mda.mda:	1	2.105007e+05
INFO:gemseo.mda.mda:	2	2.243808e+05
INFO:gemseo.mda.mda:	3	9.730448e+04
INFO:gemseo.mda.mda:	4	5.114785e+04
INFO:gemseo.mda.mda:	5	6.321384e+03
INFO:gemseo.mda.mda:

INFO:gemseo.mda.mda:	8	6.276237e+01
INFO:gemseo.mda.mda:	9	1.875499e+01
INFO:gemseo.mda.mda:	10	3.428780e-01
INFO:gemseo.mda.mda:	11	1.299422e-03
INFO:gemseo.mda.mda:	12	2.237412e-06
INFO:gemseo.mda.mda:	13	1.917468e-08
INFO:gemseo.mda.mda:	1	2.105029e+05
INFO:gemseo.mda.mda:	2	2.243830e+05
INFO:gemseo.mda.mda:	3	9.730586e+04
INFO:gemseo.mda.mda:	4	5.114870e+04
INFO:gemseo.mda.mda:	5	6.321404e+03
INFO:gemseo.mda.mda:	6	7.633523e+02
INFO:gemseo.mda.mda:	7	1.830029e+02
INFO:gemseo.mda.mda:	8	6.276227e+01
INFO:gemseo.mda.mda:	9	1.875489e+01
INFO:gemseo.mda.mda:	10	3.428463e-01
INFO:gemseo.mda.mda:	11	1.299353e-03
INFO:gemseo.mda.mda:	12	2.237360e-06
INFO:gemseo.mda.mda:	13	1.896952e-08
INFO:gemseo.mda.mda:	1	2.105029e+05
INFO:gemseo.mda.mda:	2	2.243831e+05
INFO:gemseo.mda.mda:	3	9.730579e+04
INFO:gemseo.mda.mda:	4	5.114863e+04
INFO:gemseo.mda.mda:	5	6.321429e+03
INFO:gemseo.mda.mda:	6	7.633621e+02
INFO:gemseo.mda.mda:	7	1.830030e+02
INFO:gemseo.mda.mda:	8	6.276237e+01
INFO:gemseo.mda.mda:

...

INFO:gemseo.mda.mda:	4	2.335512e+04
INFO:gemseo.mda.mda:	5	1.908216e+04
INFO:gemseo.mda.mda:	6	5.183732e+03
INFO:gemseo.mda.mda:	7	2.416899e+03
INFO:gemseo.mda.mda:	8	5.100506e+02
INFO:gemseo.mda.mda:	9	5.929038e+01
INFO:gemseo.mda.mda:	10	8.216802e+00
INFO:gemseo.mda.mda:	11	5.266861e+00
INFO:gemseo.mda.mda:	12	8.979053e-02
INFO:gemseo.mda.mda:	13	7.560656e-03
INFO:gemseo.mda.mda:	14	2.210352e-03
INFO:gemseo.mda.mda:	15	1.661300e-05
INFO:gemseo.mda.mda:	16	6.341999e-08
INFO:gemseo.mda.mda:	1	1.349548e+05
INFO:gemseo.mda.mda:	2	1.526982e+05
INFO:gemseo.mda.mda:	3	4.987817e+04
INFO:gemseo.mda.mda:	4	2.335511e+04
INFO:gemseo.mda.mda:	5	1.908218e+04
INFO:gemseo.mda.mda:	6	5.183737e+03
INFO:gemseo.mda.mda:	7	2.416893e+03
INFO:gemseo.mda.mda:	8	5.100513e+02
INFO:gemseo.mda.mda:	9	5.929148e+01
INFO:gemseo.mda.mda:	10	8.217328e+00
INFO:gemseo.mda.mda:	11	5.266773e+00
INFO:gemseo.mda.mda:	12	8.978888e-02
INFO:gemseo.mda.mda:	13	7.561437e-03
INFO:gemseo.mda.mda:	14	2.211761e-03
INFO:gemseo.mda.

INFO:gemseo.mda.mda:	5	1.908220e+04
INFO:gemseo.mda.mda:	6	5.183724e+03
INFO:gemseo.mda.mda:	7	2.416901e+03
INFO:gemseo.mda.mda:	8	5.100581e+02
INFO:gemseo.mda.mda:	9	5.929451e+01
INFO:gemseo.mda.mda:	10	8.218355e+00
INFO:gemseo.mda.mda:	11	5.266722e+00
INFO:gemseo.mda.mda:	12	8.978969e-02
INFO:gemseo.mda.mda:	13	7.563044e-03
INFO:gemseo.mda.mda:	14	2.214085e-03
INFO:gemseo.mda.mda:	15	1.664228e-05
INFO:gemseo.mda.mda:	16	6.352252e-08
INFO:gemseo.mda.mda:	1	1.349547e+05
INFO:gemseo.mda.mda:	2	1.526981e+05
INFO:gemseo.mda.mda:	3	4.987821e+04
INFO:gemseo.mda.mda:	4	2.335514e+04
INFO:gemseo.mda.mda:	5	1.908218e+04
INFO:gemseo.mda.mda:	6	5.183726e+03
INFO:gemseo.mda.mda:	7	2.416901e+03
INFO:gemseo.mda.mda:	8	5.100544e+02
INFO:gemseo.mda.mda:	9	5.929241e+01
INFO:gemseo.mda.mda:	10	8.217551e+00
INFO:gemseo.mda.mda:	11	5.266799e+00
INFO:gemseo.mda.mda:	12	8.979032e-02
INFO:gemseo.mda.mda:	13	7.561815e-03
INFO:gemseo.mda.mda:	14	2.212134e-03
INFO:gemseo.mda.mda:	15	1.662690e-05
INFO:gemseo.mda

INFO:gemseo.mda.mda:	6	4.978242e+03
INFO:gemseo.mda.mda:	7	2.350765e+03
INFO:gemseo.mda.mda:	8	5.547967e+02
INFO:gemseo.mda.mda:	9	9.884082e+01
INFO:gemseo.mda.mda:	10	2.981095e+01
INFO:gemseo.mda.mda:	11	1.008372e+00
INFO:gemseo.mda.mda:	12	2.240163e-01
INFO:gemseo.mda.mda:	13	3.937233e-03
INFO:gemseo.mda.mda:	14	1.767657e-04
INFO:gemseo.mda.mda:	15	2.581126e-06
INFO:gemseo.mda.mda:	16	4.148179e-08
INFO:gemseo.mda.mda:	1	1.323036e+05
INFO:gemseo.mda.mda:	2	1.493934e+05
INFO:gemseo.mda.mda:	3	4.847789e+04
INFO:gemseo.mda.mda:	4	2.283945e+04
INFO:gemseo.mda.mda:	5	1.893074e+04
INFO:gemseo.mda.mda:	6	4.978242e+03
INFO:gemseo.mda.mda:	7	2.350765e+03
INFO:gemseo.mda.mda:	8	5.547967e+02
INFO:gemseo.mda.mda:	9	9.884082e+01
INFO:gemseo.mda.mda:	10	2.981095e+01
INFO:gemseo.mda.mda:	11	1.008372e+00
INFO:gemseo.mda.mda:	12	2.240163e-01
INFO:gemseo.mda.mda:	13	3.937233e-03
INFO:gemseo.mda.mda:	14	1.767657e-04
INFO:gemseo.mda.mda:	15	2.581126e-06
INFO:gemseo.mda.mda:	16	4.148179e-08
INFO:gemseo.md

INFO:gemseo.core.mdo_scenario:*** MDO Scenario run terminated in 0:00:49.150256 ***
INFO:SoS.EE.SoSOptimScenario:Post run at xopt
INFO:SoS.EE.SoSOptimScenario:Executing at xopt point [6.00004666e-02 6.00000000e+04 1.40000000e+00 2.50000000e+00
 7.00000000e+01 1.50000000e+03 4.00000000e-01 7.50000000e-01
 7.50000000e-01 1.56251547e-01]
INFO:SoS.EE.SoSOptimScenario:x_opt from problem solution is [6.00004666e-02 6.00000000e+04 1.40000000e+00 2.50000000e+00
 7.00000000e+01 1.50000000e+03 4.00000000e-01 7.50000000e-01
 7.50000000e-01 1.56251547e-01]
INFO:SoS.EE.SoSOptimScenario:list of functions to evaluate [optim.SobOptimScenario.g_1(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3) <= 0.0, optim.SobOptimScenario.g_2(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3) <= 0.0, optim.SobOptimScenario.g_3(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScen

{'optim.SobOptimScenario.max_iter': array([10]), 'optim.SobOptimScenario.ineq_constraints': [1, 1, 1], 'optim.SobOptimScenario.eq_constraints': [], 'optim.SobOptimScenario.algo': 1, 'optim.SobOptimScenario.design_space': array(['z', 'x_1', 'x_2', 'x_3', 'y_14', 'y_32', 'y_31', 'y_24', 'y_34',
       'y_23', 'y_21', 'y_12',
       list([0.05, 45000.0, 1.6, 5.5, 55.0, 1000.0]), list([0.25, 1.0]),
       list([1.0]), list([0.5]), list([50606.9741711, 7306.20262124]),
       list([0.50279625]), list([6354.32430691]), list([4.15006276]),
       list([1.10754577]), list([12194.2671934]), list([50606.9741711]),
       list([50606.9742, 0.95]),
       list([0.01, 30000.0, 1.4, 2.5, 40.0, 500.0]), list([0.1, 0.75]),
       list([0.75]), list([0.1]), list([24850.0, -7700.0]), list([0.235]),
       list([2960.0]), list([0.44]), list([0.44]), list([3365.0]),
       list([24850.0]), list([24850.0, 0.45]),
       list([0.09, 60000.0, 1.8, 8.5, 70.0, 1500.0]), list([0.4, 1.25]),
       list([1.25]), 

# 6) Explore results

In [50]:
# retrieve discipline to check the result...
opt_disc = exec_eng.dm.get_disciplines_with_name("optim." + sc_name)[0]

In [51]:
opt_disc.get_disc_full_name()

'optim.SobOptimScenario'

In [52]:
DESC_IN = opt_disc.DESC_IN
DESC_OUT = opt_disc.DESC_OUT
DESC_IN_df = pd.DataFrame.from_dict(DESC_IN,orient='index')
DESC_OUT_df = pd.DataFrame.from_dict(DESC_OUT,orient='index')

In [53]:
DESC_IN_df

,type,default,structuring,possible_values,dataframe_descriptor,dataframe_edition_locked
max_iter,float,NaN,NaN,NaN,NaN,NaN
ineq_constraints,string_list,[],True,NaN,NaN,NaN
eq_constraints,string_list,[],True,NaN,NaN,NaN
algo,string,NaN,True,NaN,NaN,NaN
design_space,dataframe,NaN,True,NaN,NaN,NaN
formulation,string,NaN,True,NaN,NaN,NaN
maximize_objective,bool,False,True,NaN,NaN,NaN
objective_name,string,NaN,True,NaN,NaN,NaN
differentiation_method,string,finite_differences,True,"[user, finite_differences, complex_step]",NaN,NaN
algo_options,dict,NaN,True,NaN,"{'variable': ('string', None, False), 'value':...",False


In [54]:
DESC_OUT_df

,type
design_space_out,dataframe


In [55]:
opt_disc.get_data_io_dict('in')['formulation']['value']

'MDF'

In [56]:
opt_disc.get_data_io_dict('in')['algo']['value']

'SLSQP'

In [57]:
opt_disc.get_data_io_dict('in')['objective_name']['value']

'y_4'

In [58]:
opt_disc.optimization_result.f_opt

3963.4211565505475

In [59]:
opt_disc.optimization_result.x_opt

array([6.00004666e-02, 6.00000000e+04, 1.40000000e+00, 2.50000000e+00,
       7.00000000e+01, 1.50000000e+03, 4.00000000e-01, 7.50000000e-01,
       7.50000000e-01, 1.56251547e-01])

In [60]:
opt_disc._post_run()

INFO:SoS.EE.SoSOptimScenario:Executing at xopt point [6.00004666e-02 6.00000000e+04 1.40000000e+00 2.50000000e+00
 7.00000000e+01 1.50000000e+03 4.00000000e-01 7.50000000e-01
 7.50000000e-01 1.56251547e-01]
INFO:SoS.EE.SoSOptimScenario:x_opt from problem solution is [6.00004666e-02 6.00000000e+04 1.40000000e+00 2.50000000e+00
 7.00000000e+01 1.50000000e+03 4.00000000e-01 7.50000000e-01
 7.50000000e-01 1.56251547e-01]
INFO:SoS.EE.SoSOptimScenario:list of functions to evaluate [optim.SobOptimScenario.g_1(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3) <= 0.0, optim.SobOptimScenario.g_2(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3) <= 0.0, optim.SobOptimScenario.g_3(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, optim.SobOptimScenario.x_2, optim.SobOptimScenario.x_3) <= 0.0, -optim.SobOptimScenario.y_4(optim.SobOptimScenario.z, optim.SobOptimScenario.x_1, op

In [61]:
opt_disc.get_data_io_dict('out')['design_space_out']['value'] 

,variable,value,lower_bnd,upper_bnd,enable_variable,activated_elem
0,z,"[0.06000046664060199, 60000.0, 1.4, 2.5, 70.0,...","[0.01, 30000.0, 1.4, 2.5, 40.0, 500.0]","[0.09, 60000.0, 1.8, 8.5, 70.0, 1500.0]",True,"[True, True, True, True, True, True]"
1,x_1,"[0.4, 0.75]","[0.1, 0.75]","[0.4, 1.25]",True,"[True, True]"
2,x_2,[0.75],[0.75],[1.25],True,[True]
3,x_3,[0.15625154713123346],[0.1],[1.0],True,[True]
0,y_14,"[50606.9741711, 7306.20262124]","[24850.0, -7700.0]","[77100.0, 45000.0]",True,"[True, True]"
1,y_32,[0.50279625],[0.235],[0.795],True,[True]
2,y_31,[6354.32430691],[2960.0],[10185.0],True,[True]
3,y_24,[4.15006276],[0.44],[11.13],True,[True]
4,y_34,[1.10754577],[0.44],[1.98],True,[True]
5,y_23,[12194.2671934],[3365.0],[26400.0],True,[True]
